## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-27-18-50-03 +0000,bbc,FBI fires agents pictured kneeling at George F...,https://www.bbc.com/news/articles/cr4q5xrz9dpo...
1,2025-09-27-18-41-48 +0000,bbc,PM takes aim at 'toxic' Reform as he arrives f...,https://www.bbc.com/news/articles/cn0xzdgyx0do...
2,2025-09-27-18-39-23 +0000,nyt,Stampede at Actor’s Political Rally Kills at L...,https://www.nytimes.com/2025/09/27/world/asia/...
3,2025-09-27-18-36-08 +0000,nyt,U.S. Ambassador to Israel Is Expected to Make ...,https://www.nytimes.com/2025/09/27/world/middl...
4,2025-09-27-18-35-45 +0000,bbc,Trump authorises 'full force' as Portland beco...,https://www.bbc.com/news/articles/cddmn6ge6e2o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2375/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
32,trump,59
747,new,15
203,comey,14
84,ice,14
732,tariffs,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
147,2025-09-27-02-03-00 +0000,wsj,The Supreme Court on Friday granted the Trump ...,https://www.wsj.com/politics/policy/supreme-co...,117
127,2025-09-27-06-45-39 +0000,nypost,US to revoke Colombian president’s visa after ...,https://nypost.com/2025/09/27/us-news/us-to-re...,116
232,2025-09-26-20-38-00 +0000,wsj,President Trump announced many new tariffs Thu...,https://www.wsj.com/economy/trade/trump-to-sla...,114
11,2025-09-27-18-19-23 +0000,cbc,Trump says he's sending troops to Portland to ...,https://www.cbc.ca/news/world/trump-portland-t...,108
30,2025-09-27-16-43-23 +0000,nyt,"Trump Says He Has Ordered Troops to Portland, ...",https://www.nytimes.com/2025/09/27/us/politics...,108


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
147,117,2025-09-27-02-03-00 +0000,wsj,The Supreme Court on Friday granted the Trump ...,https://www.wsj.com/politics/policy/supreme-co...
149,58,2025-09-27-02-00-00 +0000,wsj,A U.S. plan to field thousands of cutting-edge...,https://www.wsj.com/politics/national-security...
127,57,2025-09-27-06-45-39 +0000,nypost,US to revoke Colombian president’s visa after ...,https://nypost.com/2025/09/27/us-news/us-to-re...
158,46,2025-09-27-01-00-00 +0000,wsj,The indictment of James Comey sealed the remar...,https://www.wsj.com/politics/policy/trump-over...
49,40,2025-09-27-14-47-55 +0000,nypost,Trump orders troop deployment to ‘war ravaged’...,https://nypost.com/2025/09/27/us-news/trump-or...
153,37,2025-09-27-01-25-33 +0000,latimes,He was body-slammed by immigration agents. Now...,https://www.latimes.com/california/story/2025-...
6,37,2025-09-27-18-29-53 +0000,nypost,Israel strikes 120 targets in Gaza Strip over ...,https://nypost.com/2025/09/27/world-news/israe...
21,30,2025-09-27-17-27-30 +0000,nypost,Black eye club: Howard Rubin’s shiner sparks I...,https://nypost.com/2025/09/27/us-news/howard-r...
232,30,2025-09-26-20-38-00 +0000,wsj,President Trump announced many new tariffs Thu...,https://www.wsj.com/economy/trade/trump-to-sla...
40,30,2025-09-27-15-46-19 +0000,nypost,Renaissance Faire ‘Lord Mayor’ sues ex-lover f...,https://nypost.com/2025/09/27/us-news/lady-lor...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
